In [1]:
import os
import cv2
import torch
import json
import numpy as np
from google.colab import drive
from IPython.display import clear_output 

In [2]:
import matplotlib.pyplot as plt

Скачиваем модель

In [3]:
# !gdown --id 1waoOULCmz1PA0K4pfSkTVHsCe9GYxzte -O /content/model.pt

Клонируем репу

In [4]:
%cd /content
# !git clone https://github.com/STASVLAD/Tips-and-Tricks-in-DL

/content


Устанавливаем необходимые зависимости

In [5]:
# %cd Tips-and-Tricks-in-DL
# %pip install -qr requirements.txt 
# 
# print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# убрать 
%cd /content/drive/MyDrive/Project/yolov3
%pip install -qr requirements.txt 

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

/content/drive/MyDrive/Project/yolov3
     |████████████████████████████████| 645kB 8.3MB/s 
Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


Создаем необходимые временные папки и скачиваем видео 

In [8]:
!sudo rm -rf /content/images
!sudo rm -rf /content/drive/MyDrive/Project/yolov3/runs
!sudo rm -rf /content/output
!sudo rm -rf /content/images
!sudo rm -rf /content/video.mp4

In [9]:
%cd /content
!mkdir images
!mkdir output

/content


In [10]:
# !gdown --id 1kh34S1LEW7eMB48hCsCukUXWvWgl2HEI -O /content/video.mp4
!gdown --id 1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT -O /content/video.mp4

Downloading...
From: https://drive.google.com/uc?id=1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT
To: /content/video.mp4
14.6MB [00:00, 55.2MB/s]


Преобразовываем видео в последовательность картинок

In [11]:
%%time
!ffmpeg -i '/content/video.mp4' -s 1280x720  '/content/images/out-%06d.jpg'
clear_output()

CPU times: user 104 ms, sys: 9.27 ms, total: 113 ms
Wall time: 11.5 s


Детектим светофоры с помощью подгруженной ранее модели

In [12]:
%%time
%cd /content/drive/MyDrive/Project/yolov3
!python detect_custom.py --weights /content/drive/MyDrive/Alex.pt --img 640 --source /content/images/ --save-dict > /dev/nul

/content/drive/MyDrive/Project/yolov3
YOLOv3  torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.3 GFLOPS
CPU times: user 311 ms, sys: 42.4 ms, total: 354 ms
Wall time: 1min 14s


In [13]:
def detect_color(img, Threshold=0.01):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # lower mask (0-10)
    lower_red = np.array([0, 70, 50])
    upper_red = np.array([10, 255, 255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red1 = np.array([170, 70, 50])
    upper_red1 = np.array([180, 255, 255])
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)

    # defining the Range of yellow color
    lower_yellow = np.array([21, 39, 64])
    upper_yellow = np.array([40, 255, 255])
    mask2 = cv2.inRange(img_hsv, lower_yellow, upper_yellow)

    # defining the Range of green color
    lowew_green = np.array([40, 30, 72])
    upper_green = np.array([102, 255, 255])
    mask3 = cv2.inRange(img_hsv, lowew_green, upper_green)

    # red pixels' mask
    mask_red = mask0 + mask1
    mask_yellow = mask2
    mask_green = mask3

    # Compare the percentage
    rate_red = np.count_nonzero(mask_red) / (img.shape[0] * img.shape[1])
    rate_yellow = np.count_nonzero(mask_yellow) / (img.shape[0] * img.shape[1])
    rate_green = np.count_nonzero(mask_green) / (img.shape[0] * img.shape[1])
    rates = [rate_red, rate_yellow, rate_green]
    colors = ['red', 'yellow', 'green']
    rate_max = max(rates)
    if rate_max > Threshold:
        return colors[rates.index(rate_max)]
    else:
        return 'unknown'

In [14]:
def affect_check(x_l, x_r, img_size):
    x_thr_r = 0.6 * img_size[1]
    x_thr_l = 0.4 * img_size[1]
    if  x_l >= x_thr_r or x_r <= x_thr_l:
        return True       
    else:
        return False

In [15]:
frame_boxes_path = '/content/drive/MyDrive/Project/yolov3/runs/detect/exp/boxes.json'
with open(frame_boxes_path) as json_file:
    frame_boxes = json.load(json_file)

In [16]:
img_folder = '/content/images'
img_names = os.listdir(img_folder)
img_names.sort()

In [17]:
result = {}
keys = range(len(img_names))
result = {key:{} for key in keys}

In [18]:
%%time
for img_path, boxes in frame_boxes.items():
    j = 0
    for box in boxes:
        xywh = np.array(box)
        img = cv2.imread(img_path)
        h = img.shape[0]
        w = img.shape[1]
        x1 = round((xywh[0] - xywh[2] / 2) * w)
        x2 = round((xywh[0] + xywh[2] / 2) * w)
        y1 = round((xywh[1] - xywh[3] / 2) * h)
        y2 = round((xywh[1] + xywh[3] / 2) * h)
        img_crop = img[y1:y2,x1:x2]

        color = detect_color(img_crop)
        affect = affect_check(x1, x2, img.shape)

        img_name = os.path.split(img_path)[-1]
        frame_id = img_names.index(img_name)
        result[frame_id]["traffic_light_%d" % j] = { 'coords': [x1, y1, x2, y2], 
                                                     'state': color,
                                                     'affect': affect
                                                    }
        j += 1

CPU times: user 12.1 s, sys: 95.4 ms, total: 12.2 s
Wall time: 12.1 s


In [19]:
with open('/content/drive/MyDrive/result.json', 'w') as fp:
    json.dump(result, fp, indent=4)

КОНЕЦ!

Далее сделаем видео

Размечаем картинки

In [20]:
output_folder = '/content/output'
for frame_id in result.keys():
    img_path = os.path.join(img_folder, img_names[frame_id])
    img = cv2.imread(img_path)
    for tl in result[frame_id].values():
        color = tl['state']
        box = tl['coords']
        affect = tl['affect']
        label = color + ' (' + str(affect) + ')'
        cv2.putText(img, label, (box[2], box[3]), 
                    cv2.FONT_HERSHEY_COMPLEX, 
                    fontScale=0.8, color=(128, 0, 128), thickness=2)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), 
                      color=(128, 0, 128), thickness=2)
    path_out = os.path.join(output_folder, img_names[frame_id])
    cv2.imwrite(path_out, img)

Собираем видео из размеченных картинок 

In [21]:
video = cv2.VideoCapture('/content/video.mp4');
fps = video.get(cv2.CAP_PROP_FPS)
video.release(); 
with open('/content/fps.txt', 'w') as fp:
    fp.write(str(fps))

In [22]:
%%time
%cd /content/drive/MyDrive/videos/
!ffmpeg -y -framerate "$(< /content/fps.txt)" -i /content/output/out-%06d.jpg -c:v libx264 -crf 18 -pix_fmt yuv420p "$(date)"".mp4"
clear_output()

CPU times: user 636 ms, sys: 64.8 ms, total: 701 ms
Wall time: 1min 16s
